##  hopfield network to detect english alphabet images

In [92]:
import numpy as np
from PIL import Image, ImageFont
import random
import matplotlib.image as mpimg
import os

In [95]:
def train(patterns):
    #print(patterns.shape)
    weights = np.zeros((patterns.shape[1], patterns.shape[1]))
    for p in patterns:
        for i in range(weights.shape[0]):
            for j in range(weights.shape[1]):
                if i == j:
                    weights[i, j] = 0
                else:
                    weights[i, j] += p[i] * p[j]
                    #weights[j, i] = weights[i, j]
    return weights


def sign(x):
    if x >= 0:
        return 1
    else:
        return -1



def test(input_pattern, weights):
    before_iterate = np.copy(input_pattern)
    after_iterate = np.copy(input_pattern)
    n = 0
    while True:
        n += 1
        if n % 10 == 0:
            print(n)
        before_iterate = np.copy(after_iterate)
        # print(after_iterate.shape[0])
        for idx in range(after_iterate.shape[0]):
            result = np.dot(after_iterate, weights[idx])
            #if after_iterate[idx] != sign(result):
                #print(idx, n)
                #print(after_iterate[idx], sign(result))
            after_iterate[idx] = sign(result)
        if np.array_equal(before_iterate, after_iterate) is True:
            break
        if n >= 100:
            #print("this is not going to converge.")
            break
    #print(f"number of iteration: {n}")
    #print("after_iterate", np.array_equal(after_iterate, input_pattern))
   # print(energy_function(weights, after_iterate))
   # print()
    return after_iterate



In [96]:
def energy_function(weights, vector):
    energy = 0
    energy = -sum([weights[i, j] * vector[i] * vector[j] for i in range(len(vector)) for j in range(len(vector))])
    return energy

    

In [97]:
def save_images():
    sizes = [16, 32, 64]
    for font_size in sizes:
        # create folder if not exists
        if not os.path.exists(f"image_patterns/font_size_{font_size}"):
            os.mkdir(f"image_patterns/font_size_{font_size}")
          
        # load first 10 characters
        font = ImageFont.truetype("arial.ttf", font_size)
        for char in "ABCDEFGHIJ":
            im = Image.Image()._new(font.getmask(char))
            im.save(f"image_patterns\\font_size_{font_size}\\{char}_{font_size}.bmp")

            
def generate_noisy_images():
    if not os.path.exists(f"image_noisy"):
        os.mkdir(f"image_noisy")
    sizes = [16, 32, 64]
    noise_percents = [0.1, 0.3, 0.6]
    for font_size in sizes:
        for char in "ABCDEFGHIJ":
            img = mpimg.imread(f"image_patterns\\font_size_{font_size}\\{char}_{font_size}.bmp")
            #if font_size == 16 and char == "A":
             #   print("imageeeeeeee = ", img)
            width = len(img)
            height = len(img[0])
            img_list = [-1 if img[i, j] < 20 else 1 for i in range(len(img)) for j in range(len(img[0]))]
            for percent in noise_percents:
                if not os.path.exists(f"image_noisy\\font_size_{font_size}_p{percent}"):
                    os.mkdir(f"image_noisy\\font_size_{font_size}_p{percent}")
            
                for i in range(int(percent * 200)):
                    rnd = random.randrange(1, width * height)
                    img_list[rnd] = 1 if img_list[rnd] == -1 else -1
                noisy_list = [0 if img_list[i] == 1 else 255 for i in range(len(img_list))]
                noisy_array = np.array(img_list).reshape(width, height)
                noisy_img = Image.fromarray(noisy_array.astype(np.uint8))
                noisy_img.save(f"image_noisy\\font_size_{font_size}_p{percent}\\{char}_{font_size}_p{percent}.bmp")
                #mpimg.imsave(f"image_noisy\\font_size_{font_size}_p{percent}\\{char}_{font_size}_p{percent}.bmp", noisy_array)
                #image = mpimg.imread(f"image_noisy\\font_size_{font_size}_p{percent}\\{char}_{font_size}_p{percent}.bmp")
                #print(image[0,0])
    return





def make_patterns():
    # load patterns
    font_sizes = [16]
    all_sizes_patterns = []
    
    patterns_32 = []
    patterns_64 = []
    z = 0
    for f in font_sizes:
        #print("round:", z, "font:", f)
        z += 1
        patterns = []
        for c in "ABCDEFGHIJ":
            #im = Image.open(f"{c}_16.bmp").convert('RGB')
            #px = im.load()
            img = mpimg.imread(f"image_patterns\\font_size_{f}\\{c}_{f}.bmp")
            #width, height = im.size
            p = [-1 if img[i, j] < 20 else 1 for i in range(len(img)) for j in range(len(img[0]))]
            
            for i in range(200 - len(p)):
                p.append(-1)
            patterns.append(p)
        all_sizes_patterns.append(patterns)
        #print("shape=", np.array(all_sizes_patterns).shape)
    return all_sizes_patterns


In [99]:
def hamming_dist(arr1, arr2):
    return

def evaluate():
    font_size = [16, 32, 64]
    noise_percents = [0.1, 0.3, 0.6]
    for f in font_size:
        for p in noise_percents:
            for c in "ABCDEFGHIJ":
                result = mpimg.imread(f"image_output\\font_size_{f}_p{p}\\{c}_{f}_p{p}")
                pattern = mpimg.imread(f"image_pattern\\font_size_{f}_p{p}\\{c}_{f}_p{p}")
                #print(f"accuracy of font size {f} and noist {p} is: {hamming_dist(result, pattern)}")
                

In [101]:
def main():
    save_images()
    generate_noisy_images()

    all_sizes_patterns = make_patterns()
    
    
    if not os.path.exists(f"image_output"):
        os.mkdir(f"image_output")
            
    font_sizes = [16]
    noise_percents = [0.1, .3, .6]
    #print(len(all_sizes_patterns))
    z = 0
    for f in font_sizes:
        weights = train(np.array(all_sizes_patterns[z]))
        z += 1
        for p in noise_percents:
            
            if not os.path.exists(f"image_output\\font_size_{f}_p{p}"):
                os.mkdir(f"image_output\\font_size_{f}_p{p}")
        
            for c in "ABCDEFGHIJ":
                img = mpimg.imread(f"image_noisy\\font_size_{f}_p{p}\\{c}_{f}_p{p}.bmp")
                width, height = len(img), len(img[0])
                #print(img[0,0])
                input_pattern = [1 if img[i, j] == 0 else -1 for i in range(len(img)) for j in range(len(img[0]))]
                for i in range(200 - len(input_pattern)):
                    input_pattern.append(-1)
                output = test(np.array(input_pattern), weights)
                output_list = [0 if output[i] == 1 else 255 for i in range(len(output))]
                output_array = np.array(output_list[0:width * height]).reshape(width, height)
                output_image = Image.fromarray(output_array.astype(np.uint8))
                output_image.save(f"image_output\\font_size_{f}_p{p}\\{c}_{f}_p{p}.bmp")
        #for p in all_sizes_patterns[0]:
           # print("pattern energy:", energy_function(weights, p))
if __name__ == "__main__":

    main()